In [2]:
# imports 

import numpy as np
import heapq
import math
import scipy as sp
from matplotlib import pyplot
from dataclasses import dataclass

In [71]:
# Person structure
@dataclass
class Person:
    arrival: float # time of arrival
    onboard: float # time they boarded the elevator
    dest: int      # destination
    stops: int = 0 # number of stops waited (not including their dest)

# Elevator structure

@dataclass
class Elevator:
    # ELEVATOR STATE
    clients: list  # index j is list of Persons that want to go to floor j
    path: list # of (floor number, time of arrival)
    index: int # index into elevators list
    direction: int = 0 # 1 for up, 0 for stationary, -1 for down
    size: int = 0 # amount of people inside
    # FOR PERFORMANCE METRICS
    time_last_floor: float = 0 # time of arrival at last serviced floor
    last_floor: int = 0 # last floor serviced
    distance: int = 0 # total distance traveled
    idle: float = 0 # total time spent idle
    idle_a: float = 0 # time that idle period begins
    avg_occupancy: float = 0 # average occupancy rate, accurate to time_last_floor

# Floor structure

# list of pairs (floor, time of press, direction)
# pair of last times that elevator came to service down/up
# NOTE: all new arrivals are generated at time of elevator arrival
# clients stores people who did not fit inside
# ENSURE clients[j] is sorted by time of arrival

@dataclass
# for the lists below
# 0 is dummy value, 1 is for up, 2/-1 is for down 
class Floor:
    number: int # floor number
    clients: list # list of Persons that want to go up/down
    times: list  # times of last service for up/down

# Stores the metrics 
@dataclass
class Metrics:
    avg_wait: float = 0
    avg_service: float = 0
    avg_stops: float = 0
    distance: float = 0
    occupancy: float = 0
    idle: float = 0
    distance_n: int = 0
    stops_n: int = 0 # n people avg stops before destination
    service_n: int = 0 # n people avg service
    idle_n: int = 0
    wait_n: int = 0 # n people avg wait
    occupancy_n: int = 0

In [4]:
# Scheduling policy tips and pointers

# RETINK HOW WE DETERMINE WHERE ELEVATORS ARE AT ANY POINT IN TIME...
# WE DEFINETLY NEED TO CALCULATE IT FROM PREVIOUS, EASY BUT BE AWARE

# Either
# an elvator that has same direction as button
# or a dormant elevator

# What if the optimal elevator is alredy scheduled?
# then we shouldn't change anything

# How do we handle ties?

# cases to keep in mind:
# we can insert a new first state in the path
# or somewhere in the middle for a moving elevator

# Needs to adjust time spent idle
# for dormant elevators

# e_clocks has to be updated if dormant elevator is moved
# or first state in elevator path changes

# FATEST ELEVATOR POLICY 

# if tie current "winner" is kept
# maybe other tiebreaking policies...

# elevator must be non-full to be considered

# floor as floor number not structure
def fastest_elevator(elevators, time, floor, direction, capacity, e_clocks):

    # The algorithm below "streams" in the elevators and keeps a running "best" one
    c_e_arrival_t = time  # the time of arrival to rung floor by fastest elevator
    e_ix = -1             # index of elevator to have path modified
    path_ix = -1          # the index to insert into the path
    change = False        # whether we change any path
    first = False         # augmented first state in path

    for i, elevator in enumerate(elevators):
        
        # if full, skip it
        if elevator.size == capacity:
            continue
        e_floor = elevator.last_floor # current floor of the elevator (rounded down)
        if elevator.direction == direction: # moving
            
            # current floor of elevator based on time elapsed (1 floor traveled per unit of time)
            if elevator.direction == 1:
                e_floor += np.floor(time - elevator.time_last_floor)
            else:
                e_floor -= np.floor(time - elevator.time_last_floor)

            # check to see if insert is at beginning of path
            if ((direction == 1 and 0 <= floor - e_floor and elevator.path[0][0] > floor) or \
              (direction == -1 and 0 <= e_floor - floor and elevator.path[0][0] < floor)) and  \
              elevator.path[0][1] > c_e_arrival_t:
                    c_e_arrival_t = time + np.abs(e_floor - floor)
                    e_ix = i
                    path_ix = -1
                    change = True
                    first = True

            # check insert middle of path
            for j, (e_floor, e_finish_t) in enumerate(elevator.path):
                if (direction == 1 and 0 <= floor - e_floor) or (direction == -1 and 0 <= e_floor - floor):
                    # quickest elevator is already scheduled
                    if e_floor - floor == 0 and e_finish_t < c_e_arrival_t:
                        change = False
                    # uninitalized or faster to set values
                    elif e_ix == -1 or e_finish_t + np.abs(e_floor - floor) < c_e_arrival_t:
                        c_e_arrival_t = e_finish_t + np.abs(e_floor - floor)
                        e_ix = i
                        path_ix = j
                        change = True
                        first = False
                else: # elevator is past rung floor
                    break

        elif elevator.direction == 0: # dormant
            # unitialized or closer to set values
            if e_ix == -1 or time + np.abs(e_floor - floor) < c_e_arrival_t:
                c_e_arrival_t = time + np.abs(e_floor - floor)
                e_ix = i
                path_ix = 0
                change = True

    # variables now store "closest" path information 
    # so we augment the relevant path and update clocks
    if change:
        elevators[e_ix].path.insert(path_ix + 1, (floor, c_e_arrival_t)) # augment path
        # dormant elevator to move (update idle time)
        if elevators[e_ix].direction == 0:
            elevators[e_ix].direction = direction
            heapq.heappush(e_clocks, [c_e_arrival_t, e_ix, floor])
            elevators[e_ix].idle += time - elevators[e_ix].idle_a
        # need to change e_clocks because earliest state changed
        elif first:
            for i, [_, k, __] in enumerate(e_clocks):
                if k == e_ix:
                    e_clocks[i][0] = c_e_arrival_t
                    e_clocks[i][2] = floor
                    break

In [62]:
# rv is a wrapper for some random variable/process
# alpha is the confidence level
# c is the relative error
# pages 11 - 13 of the Alexopoulos and Seila output analysis pdf
# delta(k, alpha) defined on page 11
def add(metric1, metric2):
    metric1.distance += metric2.distance
    metric1.occupancy += metric2.occupancy
    metric1.avg_stops += metric2.avg_stops
    metric1.avg_wait += metric2.avg_wait
    metric1.avg_service += metric2.avg_service
    metric1.idle += metric2.idle
    return metric1

def sub(metric1, metric2):
    metric1.distance -= metric2.distance
    metric1.occupancy -= metric2.occupancy
    metric1.avg_stops -= metric2.avg_stops
    metric1.avg_wait -= metric2.avg_wait
    metric1.avg_service -= metric2.avg_service
    metric1.idle -= metric2.idle
    return metric1

def div(metric1, i):
    metric1.distance /= (i + 1)
    metric1.occupancy /= (i + 1)
    metric1.avg_stops /= (i + 1)
    metric1.avg_wait /= (i + 1)
    metric1.avg_service /= (i + 1)
    metric1.idle /= (i + 1)
    return metric1

def update(s_vars, m1, m2, i):
        s_vars[0] = (1 - 1/i) * s_vars[0] + (i + 1) * ((m1.avg_wait - m2.avg_wait) ** 2)
        s_vars[1] = (1 - 1/i) * s_vars[1] + (i + 1) * ((m1.avg_service - m2.avg_service) ** 2)
        s_vars[2] = (1 - 1/i) * s_vars[2] + (i + 1) * ((m1.avg_stops - m2.avg_stops) ** 2)
        s_vars[3] = (1 - 1/i) * s_vars[3] + (i + 1) * ((m1.distance - m2.distance) ** 2)
        s_vars[4] = (1 - 1/i) * s_vars[4] + (i + 1) * ((m1.idle - m2.idle) ** 2)
        s_vars[5] = (1 - 1/i) * s_vars[5] + (i + 1) * ((m1.occupancy - m2.occupancy) ** 2)

def update_cond(s_vars, width, m1, i, condition):
    interval = sp.stats.t.ppf(width, i - 1) * np.sqrt(s_vars) / np.sqrt(i)
    if m1.avg_wait != 0:
        condition[0] = (sp.stats.t.ppf(width, i - 1) * np.sqrt(s_vars[0]) / np.sqrt(i) ) / np.abs(m1.avg_wait)
    condition[1] = (sp.stats.t.ppf(width, i - 1) * np.sqrt(s_vars[1]) / np.sqrt(i) ) / np.abs(m1.avg_service)
    condition[2] = (sp.stats.t.ppf(width, i - 1) * np.sqrt(s_vars[2]) / np.sqrt(i) ) / np.abs(m1.avg_stops)
    condition[3] = (sp.stats.t.ppf(width, i - 1) * np.sqrt(s_vars[3]) / np.sqrt(i) ) / np.abs(m1.distance)
    condition[4] = (sp.stats.t.ppf(width, i - 1) * np.sqrt(s_vars[4]) / np.sqrt(i) ) / np.abs(m1.idle)
    condition[5] = (sp.stats.t.ppf(width, i - 1) * np.sqrt(s_vars[5]) / np.sqrt(i) ) / np.abs(m1.occupancy)

    
def relative_CI(rv, alpha, c, m, n, lambdas, capacity, h, SCHEDULER, LOADER):
    c_ = c / (1 + c)                  # c' value
    i = 1                             # ith iteration 
    s_mean_prev = Metrics()                  # (i - 1)th sample mean
    s_mean = rv(m, n, lambdas, capacity, h, SCHEDULER, LOADER)                     # ith sample mean
    # organized in metrics order
    s_vars = [0,0,0,0,0,0]                         # ith sample variance
    width = .5 + (1 - alpha) / 2      # width of interval about .5

    condition = [0,0,0,0,0,0]     

    # Calculating sample mean/var 10 iterations
    while i <= 10:
        s_mean_prev = s_mean
        # s_mean = s_mean + (rv(m) - s_mean) / (i + 1)
        s_mean = add(s_mean, div( sub(rv(m, n, lambdas, capacity, h, SCHEDULER, LOADER), s_mean), i + 1))

        #s_var = (1 - 1/i) * s_var + (i + 1) * ((s_mean - s_mean_prev) ** 2)
        update(s_vars, s_mean, s_mean_prev, i )
        i += 1

    update_cond(s_vars, width, s_mean, i, condition)

    # min inerval s_mean pairs
    
    # Stopping condition
    while(max(condition) > c_):
        s_mean_prev = s_mean
        # s_mean = s_mean + (rv(m) - s_mean) / (i + 1)
        s_mean = add(s_mean, div( sub(rv(m, n, lambdas, capacity, h, SCHEDULER, LOADER), s_mean), i + 1))

        #s_var = (1 - 1/i) * s_var + (i + 1) * ((s_mean - s_mean_prev) ** 2)
        update(s_vars, s_mean, s_mean_prev, i )

        update_cond(s_vars, width, s_mean, i, condition)
        i = i + 1
        
    return s_mean, i

In [5]:
# Loading policy tips and pointers


# First come first serve loading
# floor structure
def simple_load(elevator, lambdas, floor, time, metrics, max_capacity, e_clocks, b_clocks):

    rng = np.random.default_rng() # rv generation

    ## CLIENTS INSIDE THE ELEVATOR
    # update occupancy and distance metric
    elevator.avg_occupancy = elevator.avg_occupancy * elevator.time_last_floor / time \
         + (time - elevator.time_last_floor) * elevator.size / (time * max_capacity)
    elevator.distance += np.abs(floor.number - elevator.last_floor)
    
    # Unload people and update average service time and stops
    elevator.size -= len(elevator.clients[floor.number])
    for person in elevator.clients[floor.number]:
        metrics.avg_service = metrics.avg_service + (time - person.onboard - metrics.avg_service) / (metrics.service_n + 1)
        metrics.service_n += 1
        metrics.avg_stops = metrics.avg_stops + (person.stops - metrics.avg_stops) / (metrics.stops_n + 1)
        metrics.stops_n += 1
    elevator.clients[floor.number] = [] # clear people from elevator

    # update people's stops waited
    for _floor in elevator.clients:
        for person in _floor:
            person.stops += 1

    ## LOAD EXISTING CLIENTS INTO ELEVATOR

    # Load people in queue leftover from previous elevator arrival at this floor and update average waiting time
    i = 0 # used to "remove" people from floor queue
    for person in floor.clients[elevator.direction]:
        if elevator.size == max_capacity: # too many people
            break
        metrics.avg_wait = metrics.avg_wait + (time - person.arrival - metrics.avg_wait) / (metrics.wait_n + 1)
        metrics.wait_n += 1
        person.onboard = time
        elevator.clients[person.dest].append(person)
        elevator.size += 1
        i += 1

    # clear people loaded from floor queue using FIFO policy
    floor.clients[elevator.direction] = floor.clients[elevator.direction][i:]

    ## LOAD NEW ARRIVALS RETROSPECTIVELY (IF ANY)

    # Some variable updates
    elevator.last_floor = floor.number
    elevator.time_last_floor = time

    # You arrive "early" before the first button press
    if time - floor.times[elevator.direction] < 0:
        elevator.path = elevator.path[1:]
        # no more places to go
        if len(elevator.path) == 0:
            elevator.direction = 0
            elevator.idle_a = time
        else: # update e_clock
            heapq.heappush(e_clocks, [elevator.path[0][1], elevator.index, elevator.path[0][0]])
        return

    # "End of the line" down elevator reach 0, up elevator reach top
    if (elevator.direction == -1 and floor.number == 0) or \
        (elevator.direction == 1 and floor.number == len(elevator.clients) - 1):
        elevator.direction = 0 # should have automatically emptied out
        elevator.idle_a = time # start idling
        return

    # make cumulative sum buckets
    temp = np.array(lambdas[floor.number]) # cumuluative lambdas for the floor
    # cumulative lambdas going down (or going up)
    total = sum(lambdas[floor.number][:floor.number]) \
            if elevator.direction == -1 else \
            sum(lambdas[floor.number][floor.number:]) 
    temp = temp / total # probabilities p_ij of going from floor i to j
    # cumulative probabilities for arrivals going to destination floor j
    buckets = np.cumsum(temp[:floor.number]) \
              if elevator.direction == -1 else \
              np.cumsum(temp[floor.number:])    


    # generate arrival times 
    num_arrivals = rng.poisson((time - floor.times[elevator.direction]) * total)
    # if there are no previous people waiting add one; the person who rang
    if len(floor.clients[elevator.direction]) == 0:
        num_arrivals += 1
    arrival_times = rng.uniform(floor.times[elevator.direction], time, num_arrivals)
    arrival_times = sorted(arrival_times) 


    # Retrospective generation 
    for i in range(num_arrivals):
        U = rng.uniform(0,1) # generate probability
        # Find corresponding floor
        p = Person(arrival_times[i], arrival_times[i], 0)
        for ix, val in enumerate(buckets):
            if U <= val:
                p.dest = ix + floor.number \
                         if elevator.direction == 1 else \
                         ix
                # load into elevator if can fit inside
                if elevator.size < max_capacity:
                    elevator.clients[p.dest].append(p)
                    elevator.size += 1
                else: # elevator full
                    floor.clients[elevator.direction].append(p) # clients join queue
                break
    
    # update variables and b_clocks
    floor.times[elevator.direction] = time
    if len(floor.clients[elevator.direction]) > 0: # people still waiting so button gets pressed immediately
        heapq.heappush(b_clocks, (time, floor.number, elevator.direction))
        # infinite loop... it should be past it
    else: # generate next arrival
        heapq.heappush(b_clocks, (time + rng.exponential(1/total), floor.number, elevator.direction))
    
        
    # update elevator path
    new_path = []
    if elevator.direction == 1:
        for ix in range(1, len(elevator.clients) - floor.number):
            if (len(elevator.clients[ix + floor.number])) > 0:
                new_path.append([floor.number + ix, time + ix])
    else:
        for ix in range(floor.number)[::-1]:
            if (len(elevator.clients[ix])) > 0:
                new_path.append([ix, time + floor.number - ix])
    elevator.path = new_path

    # update e_clocks
    heapq.heappush(e_clocks, [elevator.path[0][1], elevator.index, elevator.path[0][0]])

In [57]:
#################### PARAMETERS ####################

# m         : the number of floors in the building
# n         : the number of elevators 
# lambdas   : m * m array; i,j is the rate of arrivals on floor i who want to go to floor j
# capacity  : max capacity of an elevator
# h         : length of simulation
# SCHEDULER : given list of elevator structures, (time, direction, floor of button press), and elevator clocks
#             augments an elevator structure and e_clocks
# LOADER    : given a floor and elevator stuctures
#             updates floor and elevator structures aftder unload and load

#################### SIMULATION DESIGN ####################

# Elevators have predetermined paths
# On button press, scheduler decides which paths change
# When elevator reaches a floor people get off 
# Arrivals are generated retrospectively with a queue to store excess

# Button timings are events on clock and so are elevator arrivals

#################### NOTES ####################

# Floors are labeled 0 ... m - 1
# Assume time to travel between floors is constant
# We'll assume loading happens in 0 time i.e. instantly

# All 2(m-1) buttons are initialized to begin 
# then buttons are pressed after elevator leaves the floor

#################### DATA STRUCTURES ####################

# down  : tuple where index j is rate of clients of floor j going down
# total : tuple where index j is sum of all rates

# elevators : list of elevator structures
# floors    : list of floor structures

# e_clocks : list of [time, elevator index, floor] that represent the next stop for all active elevators
# b_clocks : list of (time, floor number, direction) that represent floor button presses

#################### METRICS ####################
# average service and wait times 
# average capacity 
# average number of stops before destination
# elevator distance traveled
# elevator time spent idle

#################### Dictionary ####################
# c  |-> closest
# e  |-> elevator
# ix |-> index
# t  |-> time
# b  |-> button

# assuming uniform lambda on all floors 

def simple_sim(m: int, n: int, lambdas: tuple[tuple[float]], capacity: int, h: int, SCHEDULER, LOADER):

    # Initialization 
    rng       = np.random.default_rng()
    t         = 0  # total elapsed time
    e_clocks  = [] # clocks for elevator arrivals
    b_clocks  = [] # clocks for the button presses
    floors    = list( Floor(i, [0, [], []], [0]) for i in range(m) )
    elevators = list( Elevator([[] for _ in range(m)], [], i) for i in range(n) )
    down      = tuple( sum(lambdas[j][:j]) for j in range(m) )
    total     = sum(lambdas[0])
    metrics   = Metrics()

    # initialize b_clocks with both up and down presses
    for i in range(1, m-1):
        first = [rng.exponential(1/total), i]
        second = []
        if rng.uniform(0, 1) <= down[i] / total:
            first.append(-1)
            second.append(first[0] + rng.exponential(1/(total - down[i])))
            second.append(i)
            second.append(1)
            floors[i].times.append(second[0])
            floors[i].times.append(first[0])
        else:
            first.append(1)
            second.append(first[0] + rng.exponential(1/down[i]))
            second.append(i)
            second.append(-1)
            floors[i].times.append(first[0])
            floors[i].times.append(second[0])
        b_clocks.append(tuple(first))
        b_clocks.append(tuple(second))
    
    # add first and last floor buttons
    temp = rng.exponential(1/total)
    b_clocks.append((temp, 0, 1))
    floors[0].times.append(temp)
    temp = rng.exponential(1/total)
    b_clocks.append((temp, m-1, -1))
    floors[m-1].times.append(0)
    floors[m-1].times.append(temp)
    heapq.heapify(b_clocks)
    
    # Finite horizon
    while(t < h):

        if e_clocks == [] or (b_clocks != [] and b_clocks[0][0] <= e_clocks[0][0]): # if no elevator arrival or next event is button arrival
            # SCHEDULER SECTION
            time, floor, direction = heapq.heappop(b_clocks)
            t = time
            SCHEDULER(elevators, time, floor, direction, capacity, e_clocks)

        elif b_clocks == [] or b_clocks[0][0] >= e_clocks[0][0]:
            # LOADING SECTION
            time, e_ix, f_ix = heapq.heappop(e_clocks)
            t = time 
            LOADER(elevators[e_ix], lambdas, floors[f_ix], time, metrics, capacity, e_clocks, b_clocks)
        
    # return out idle and distance
    for elevator in elevators:
        metrics.idle = metrics.idle + (elevator.idle - metrics.idle) / (metrics.idle_n + 1)
        metrics.idle_n += 1
        metrics.distance = metrics.distance + (elevator.distance - metrics.distance) / (metrics.distance_n + 1)
        metrics.distance_n += 1
        metrics.occupancy = metrics.occupancy + (elevator.avg_occupancy - metrics.occupancy) / (metrics.occupancy_n + 1)
        metrics.occupancy_n += 1
    return metrics
    

In [73]:
m = 50
n = 20
capacity = 100
h = 400

relative_CI(simple_sim, .01, .01, m, n, genLambdas(m), capacity, h, fastest_elevator, simple_load)

(Metrics(avg_wait=75.43561950548533, avg_service=47.35274395401787, avg_stops=3.82040644895575, distance=305.3166666666667, occupancy=0.06339666775078165, idle=81.8585917931386, distance_n=20, stops_n=2178, service_n=2178, idle_n=20, wait_n=110, occupancy_n=20),
 11)

In [74]:
# Loading policy tips and pointers


# First come first serve loading
# floor structure
def LIFO_load(elevator, lambdas, floor, time, metrics, max_capacity, e_clocks, b_clocks):

    rng = np.random.default_rng() # rv generation

    ## CLIENTS INSIDE THE ELEVATOR
    # update occupancy and distance metric
    elevator.avg_occupancy = elevator.avg_occupancy * elevator.time_last_floor / time \
         + (time - elevator.time_last_floor) * elevator.size / (time * max_capacity)
    elevator.distance += np.abs(floor.number - elevator.last_floor)
    
    # Unload people and update average service time and stops
    elevator.size -= len(elevator.clients[floor.number])
    for person in elevator.clients[floor.number]:
        metrics.avg_service = metrics.avg_service + (time - person.onboard - metrics.avg_service) / (metrics.service_n + 1)
        metrics.service_n += 1
        metrics.avg_stops = metrics.avg_stops + (person.stops - metrics.avg_stops) / (metrics.stops_n + 1)
        metrics.stops_n += 1
    elevator.clients[floor.number] = [] # clear people from elevator

    # update people's stops waited
    for _floor in elevator.clients:
        for person in _floor:
            person.stops += 1

    ## LOAD EXISTING CLIENTS INTO ELEVATOR

    

    ## LOAD NEW ARRIVALS RETROSPECTIVELY (IF ANY)

    # Some variable updates
    elevator.last_floor = floor.number
    elevator.time_last_floor = time

    # You arrive "early" before the first button press
    if time - floor.times[elevator.direction] < 0:
        elevator.path = elevator.path[1:]
        # no more places to go
        if len(elevator.path) == 0:
            elevator.direction = 0
            elevator.idle_a = time
        else: # update e_clock
            heapq.heappush(e_clocks, [elevator.path[0][1], elevator.index, elevator.path[0][0]])
        return

    # "End of the line" down elevator reach 0, up elevator reach top
    if (elevator.direction == -1 and floor.number == 0) or \
        (elevator.direction == 1 and floor.number == len(elevator.clients) - 1):
        elevator.direction = 0 # should have automatically emptied out
        elevator.idle_a = time # start idling
        return

    # make cumulative sum buckets
    temp = np.array(lambdas[floor.number]) # cumuluative lambdas for the floor
    # cumulative lambdas going down (or going up)
    total = sum(lambdas[floor.number][:floor.number]) \
            if elevator.direction == -1 else \
            sum(lambdas[floor.number][floor.number:]) 
    temp = temp / total # probabilities p_ij of going from floor i to j
    # cumulative probabilities for arrivals going to destination floor j
    buckets = np.cumsum(temp[:floor.number]) \
              if elevator.direction == -1 else \
              np.cumsum(temp[floor.number:])    


    # generate arrival times 
    num_arrivals = rng.poisson((time - floor.times[elevator.direction]) * total)
    # if there are no previous people waiting add one; the person who rang
    if len(floor.clients[elevator.direction]) == 0:
        num_arrivals += 1
    arrival_times = rng.uniform(floor.times[elevator.direction], time, num_arrivals)
    arrival_times = sorted(arrival_times) 


    # Retrospective generation 
    for i in range(num_arrivals):
        U = rng.uniform(0,1) # generate probability
        # Find corresponding floor
        p = Person(arrival_times[i], arrival_times[i], 0)
        for ix, val in enumerate(buckets):
            if U <= val:
                p.dest = ix + floor.number \
                         if elevator.direction == 1 else \
                         ix
                # load into elevator if can fit inside
                if elevator.size < max_capacity:
                    elevator.clients[p.dest].append(p)
                    elevator.size += 1
                else: # elevator full
                    floor.clients[elevator.direction].append(p) # clients join queue
                break
    



    # Load people in queue leftover from previous elevator arrival at this floor and update average waiting time
    i = 0 # used to "remove" people from floor queue
    for person in floor.clients[elevator.direction]:
        if elevator.size == max_capacity: # too many people
            break
        metrics.avg_wait = metrics.avg_wait + (time - person.arrival - metrics.avg_wait) / (metrics.wait_n + 1)
        metrics.wait_n += 1
        person.onboard = time
        elevator.clients[person.dest].append(person)
        elevator.size += 1
        i += 1

    # clear people loaded from floor queue using FIFO policy
    floor.clients[elevator.direction] = floor.clients[elevator.direction][i:]

    # DONE WITH EVERYTHING ELSE 
    # update variables and b_clocks
    floor.times[elevator.direction] = time
    if len(floor.clients[elevator.direction]) > 0: # people still waiting so button gets pressed immediately
        heapq.heappush(b_clocks, (time, floor.number, elevator.direction))
        # infinite loop... it should be past it
    else: # generate next arrival
        heapq.heappush(b_clocks, (time + rng.exponential(1/total), floor.number, elevator.direction))
    
        
    # update elevator path
    new_path = []
    if elevator.direction == 1:
        for ix in range(1, len(elevator.clients) - floor.number):
            if (len(elevator.clients[ix + floor.number])) > 0:
                new_path.append([floor.number + ix, time + ix])
    else:
        for ix in range(floor.number)[::-1]:
            if (len(elevator.clients[ix])) > 0:
                new_path.append([ix, time + floor.number - ix])
    elevator.path = new_path

    # update e_clocks
    heapq.heappush(e_clocks, [elevator.path[0][1], elevator.index, elevator.path[0][0]])

In [ ]:
# Loading policy tips and pointers


# First come first serve loading
# floor structure
def RANDOM_load(elevator, lambdas, floor, time, metrics, max_capacity, e_clocks, b_clocks):

    rng = np.random.default_rng() # rv generation

    ## CLIENTS INSIDE THE ELEVATOR
    # update occupancy and distance metric
    elevator.avg_occupancy = elevator.avg_occupancy * elevator.time_last_floor / time \
         + (time - elevator.time_last_floor) * elevator.size / (time * max_capacity)
    elevator.distance += np.abs(floor.number - elevator.last_floor)
    
    # Unload people and update average service time and stops
    elevator.size -= len(elevator.clients[floor.number])
    for person in elevator.clients[floor.number]:
        metrics.avg_service = metrics.avg_service + (time - person.onboard - metrics.avg_service) / (metrics.service_n + 1)
        metrics.service_n += 1
        metrics.avg_stops = metrics.avg_stops + (person.stops - metrics.avg_stops) / (metrics.stops_n + 1)
        metrics.stops_n += 1
    elevator.clients[floor.number] = [] # clear people from elevator

    # update people's stops waited
    for _floor in elevator.clients:
        for person in _floor:
            person.stops += 1

    ## LOAD EXISTING CLIENTS INTO ELEVATOR

    

    ## LOAD NEW ARRIVALS RETROSPECTIVELY (IF ANY)

    # Some variable updates
    elevator.last_floor = floor.number
    elevator.time_last_floor = time

    # You arrive "early" before the first button press
    if time - floor.times[elevator.direction] < 0:
        elevator.path = elevator.path[1:]
        # no more places to go
        if len(elevator.path) == 0:
            elevator.direction = 0
            elevator.idle_a = time
        else: # update e_clock
            heapq.heappush(e_clocks, [elevator.path[0][1], elevator.index, elevator.path[0][0]])
        return

    # "End of the line" down elevator reach 0, up elevator reach top
    if (elevator.direction == -1 and floor.number == 0) or \
        (elevator.direction == 1 and floor.number == len(elevator.clients) - 1):
        elevator.direction = 0 # should have automatically emptied out
        elevator.idle_a = time # start idling
        return

    # make cumulative sum buckets
    temp = np.array(lambdas[floor.number]) # cumuluative lambdas for the floor
    # cumulative lambdas going down (or going up)
    total = sum(lambdas[floor.number][:floor.number]) \
            if elevator.direction == -1 else \
            sum(lambdas[floor.number][floor.number:]) 
    temp = temp / total # probabilities p_ij of going from floor i to j
    # cumulative probabilities for arrivals going to destination floor j
    buckets = np.cumsum(temp[:floor.number]) \
              if elevator.direction == -1 else \
              np.cumsum(temp[floor.number:])    


    # generate arrival times 
    num_arrivals = rng.poisson((time - floor.times[elevator.direction]) * total)
    # if there are no previous people waiting add one; the person who rang
    if len(floor.clients[elevator.direction]) == 0:
        num_arrivals += 1
    arrival_times = rng.uniform(floor.times[elevator.direction], time, num_arrivals)
    arrival_times = sorted(arrival_times) 


    # Retrospective generation 
    for i in range(num_arrivals):
        U = rng.uniform(0,1) # generate probability
        # Find corresponding floor
        p = Person(arrival_times[i], arrival_times[i], 0)
        for ix, val in enumerate(buckets):
            if U <= val:
                p.dest = ix + floor.number \
                         if elevator.direction == 1 else \
                         ix
                # load into elevator if can fit inside
                floor.clients[elevator.direction].append(p) # clients join queue
                break
    
    num_fit = max_capacity - elevator.size

    while num_fit > 0 and len(floor.clients[elevator.direction]) > 0:
        U = np.floor(rng.uniform(0, len(floor.clients[elevator.direction])))
        metrics.avg_wait = metrics.avg_wait + (time - floor.clients[elevator.direction][U].arrival - metrics.avg_wait) / (metrics.wait_n + 1)
        metrics.wait_n += 1
        floor.clients[elevator.direction][U].onboard = time
        elevator.clients[floor.clients[elevator.direction][U].dest].append(person)
        elevator.size += 1
        num_fit -= 1
        del floor.clients[elevator.direction][U]

    # DONE WITH EVERYTHING ELSE 
    # update variables and b_clocks
    floor.times[elevator.direction] = time
    if len(floor.clients[elevator.direction]) > 0: # people still waiting so button gets pressed immediately
        heapq.heappush(b_clocks, (time, floor.number, elevator.direction))
        # infinite loop... it should be past it
    else: # generate next arrival
        heapq.heappush(b_clocks, (time + rng.exponential(1/total), floor.number, elevator.direction))
    
        
    # update elevator path
    new_path = []
    if elevator.direction == 1:
        for ix in range(1, len(elevator.clients) - floor.number):
            if (len(elevator.clients[ix + floor.number])) > 0:
                new_path.append([floor.number + ix, time + ix])
    else:
        for ix in range(floor.number)[::-1]:
            if (len(elevator.clients[ix])) > 0:
                new_path.append([ix, time + floor.number - ix])
    elevator.path = new_path

    # update e_clocks
    heapq.heappush(e_clocks, [elevator.path[0][1], elevator.index, elevator.path[0][0]])

In [78]:
l = [-1,-2,-3,-4]
del l[2]
l

[-1, -2, -4]

In [8]:
# generate the lambda matrix 
def genLambdas(m, arrival = .2):

    lambdas = [[arrival / (m - 1) for _ in range(m)]]
    lambdas[0][0] = 0

    for i in range(1, m):
        row = []
        for j in range(m):
            if i == j:
                row.append(0.0)
            elif j == 0:
                row.append(arrival * .8)
            else:
                row.append(arrival * .2 / (m - 2) )
        lambdas.append(row)
    return lambdas

In [1]:
#link to generating poisson batch arrival: https://peters-research.com/index.php/papers/a-systematic-methodology-for-the-generation-of-lift-passengers-under-a-poisson-batch-arrival-process/
#A study on the arrival process of lift passengers in a multi-storey office building (Can be found on hunter lib webite or ask me to email it)
# the batch arrivals follow a time inhomogeneous Poisson process with piecewise constant arrival rates -> might get too complicated

#Poisson rate lambda = 0.2 in most literature. 
# The paper I mentioned above mentions that the time inhomogeneous Poisson process with piecewise constant arrival rates is a better measure for batch arrivals.


#Elevator capacity on avergae is 1.5sqft per person: https://www.tkelevator.com/us-en/company/insights/how-is-elevator-capacity-calculated.html

def max_elevator_capacity(elevator_volume: float):
    return elevator_volume/1.5


In [ ]:
# takes 2D matrix of lamda_i,j and returns a tuple containing lambda_i,down and lambda_i,up for floor i
def lambda_direction(i: int, lambdas):
    lambda_down = 0
    lambda_up = 0
    for dest in range (i):
        lambda_down += lambdas[i][dest]
    
    for dest in range (i+1, m):
        lambda_up += lambdas[i][dest]
    
    return lambda_down, lambda_up
    

In [ ]:
#tracking the clocks of button change of every floor
# lambdas is m x m matrix of lambda_i,j where i is the floor of origin and j is the destination

def DES_engine(m: int, lambda_param, h: int, lambdas):
    rng = np.random.default_rng() # rv generator
    t = 0.0
    n = 0
    generate_tuples = lambda m: [(0, 0) for _ in range(m)]
    generate_clocks = lambda m: [rng.exponential(1/lambda_param) for _ in range(m)]
    floor_states = generate_tuples(m) #initial state of every floor is (0,0)
    button_arrival = generate_clocks(m) #generates arrival times for each button on each floor, maybe have to make it a tuple for up and down ?
    
    SOF = [button_arrival, floor_states, q_up, q_down]
    
    while (t<h):
        next = min(button_arrival)
        tau = next[0]
        floor = button_arrival.index(tau)
        for i in range (m):
            
            # queues for floor i
            q_down = 0
            q_up = 0

            # generate buckets for retrospective up/down on floor i
            lambda_i = lambda_direction(i, lambdas) # stores the tuple of lambda_down and lambda_up
            p_down = lambda_i[0]/sum(lambda_i) # probability of going down

            if rng.uniform(0,1) <= p_down:
                q_down += 1
            else:
                q_up += 1
            
            # generate buckets for retrospective destination for each person in queue
            # this part requires an elevator list where elevator[j] is number of people going to floor j
            j_down = []
            j_up = []
            for j in range (0, m):
                p_ij = lambdas[i][j]/lambda_i[0] # probability of going from floor i to floor j
                j_down.append(p_ij + sum(j_down))
            
            for j in range (0, m):
                p_ij = lambdas[i][j]/lambda_i[1] # probability of going from floor i to floor j
                j_up.append(p_ij + sum(j_up))

            for q in range (q_down):
                u = rng.uniform(0,1)
                for j in j_down:
                    if u <= j_down[j]:
                        elevator[j] += 1 # one person added to elevator going to floor j (j less than i)
                        break
            
            for q in range (q_up):
                u = rng.uniform(0,1)
                for j in j_up:
                    if u <= j_up[j]:
                        elevator[i+j] += 1 # one person added to elevator going to floor i+j (j less than i)
                        break

            
                